In [1]:
%cd ..

/Users/yuqingwang/Desktop/Time Series Final


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import statsmodels.api as sm
from scipy.optimize import minimize

In [3]:
def load_data(file):
    Stock = pd.read_csv(file)
    Stock['Date'] = pd.to_datetime(Stock['Date'])
    Stock.set_index(['Date'],inplace=True)
    Stock = Stock['Close']
    Stock_return = np.log(Stock/Stock.shift())
    Stock_return.dropna(inplace = True)
    return Stock_return

In [4]:
EA_return = load_data('data/EA.csv')

In [5]:
EA_return_mean = EA_return.loc[EA_return.index<pd.to_datetime('2019-10-30')].mean()

## Constant Volatiltiy

In [6]:
vol = np.std(EA_return.loc[EA_return.index<pd.to_datetime('2019-10-30')])
vol

0.0197214701554294

## ARCH(1)

In [7]:
parameter_ARCH = np.array([vol,0.3,EA_return_mean])#alfa_0,alfa_1,miu


def L_ARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in EA_return.loc[EA_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[2])*(i-parameter_0[2])/(sigma2)/2
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[2])*(i-parameter_0[2])
    return result

res_ARCH = minimize(L_ARCH, parameter_ARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_ARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2439.547249
         Iterations: 274
         Function evaluations: 515


 final_simplex: (array([[0.00030705, 0.23172772, 0.00034225],
       [0.00030705, 0.23172772, 0.00034225],
       [0.00030705, 0.23172772, 0.00034225],
       [0.00030705, 0.23172772, 0.00034225]]), array([-2439.54724852, -2439.54724852, -2439.54724852, -2439.54724852]))
           fun: -2439.5472485239416
       message: 'Optimization terminated successfully.'
          nfev: 515
           nit: 274
        status: 0
       success: True
             x: array([0.00030705, 0.23172772, 0.00034225])

In [8]:
ARCH_estimated_vol = []
sigma2 = vol*vol
for i in EA_return:
    sigma2 = res_ARCH.x[0]+res_ARCH.x[1]*(i-res_ARCH.x[2])*(i-res_ARCH.x[2])
    ARCH_estimated_vol.append(sigma2)
ARCH_estimated_vol.pop
ARCH_estimated_vol = pd.DataFrame(ARCH_estimated_vol, index = EA_return.index, columns = ['ARCH_estimated_vol'])

In [19]:
#np.sqrt(ARCH_estimated_vol*252).to_pickle('Estimated Volatility/EA_ARCH_estimated_vol.pkl')

## GARCH(1,1)

In [10]:
parameter_GARCH = np.array([vol,0.34,0.66,EA_return_mean])#alfa_0,alfa_1,beta_1,miu


def L_GARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in EA_return.loc[EA_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[3])*(i-parameter_0[3])/(sigma2)/2
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[2]*sigma2
    return result

res_GARCH = minimize(L_GARCH, parameter_GARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_GARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2419.484337
         Iterations: 580
         Function evaluations: 983


 final_simplex: (array([[ 7.58034113e-04, -4.36918035e-03, -9.78346933e-01,
         2.60646263e-04],
       [ 7.58034114e-04, -4.36918032e-03, -9.78346933e-01,
         2.60646259e-04],
       [ 7.58034114e-04, -4.36918030e-03, -9.78346933e-01,
         2.60646244e-04],
       [ 7.58034113e-04, -4.36918032e-03, -9.78346933e-01,
         2.60646246e-04],
       [ 7.58034114e-04, -4.36918032e-03, -9.78346933e-01,
         2.60646250e-04]]), array([-2419.48433723, -2419.48433723, -2419.48433723, -2419.48433723,
       -2419.48433723]))
           fun: -2419.4843372305736
       message: 'Optimization terminated successfully.'
          nfev: 983
           nit: 580
        status: 0
       success: True
             x: array([ 7.58034113e-04, -4.36918035e-03, -9.78346933e-01,  2.60646263e-04])

In [11]:
GARCH_estimated_vol = []
sigma2 = vol*vol
for i in EA_return:
    sigma2 = res_GARCH.x[0]+res_GARCH.x[1]*(i-res_GARCH.x[3])*(i-res_GARCH.x[3])+res_GARCH.x[2]*sigma2
    GARCH_estimated_vol.append(sigma2)
GARCH_estimated_vol.pop
GARCH_estimated_vol = pd.DataFrame(GARCH_estimated_vol, index = EA_return.index, columns = ['GARCH_estimated_vol'])
GARCH_estimated_vol.mean()

GARCH_estimated_vol    0.000382
dtype: float64

In [20]:
#np.sqrt(GARCH_estimated_vol*252).to_pickle('Estimated Volatility/EA_GARCH_estimated_vol.pkl')

## TGARCH

In [13]:
parameter_TGARCH = np.array([0,0.35,0.5,0,0])#alfa_0,alfa_1,beta_!,miu,gamma1

def L_TGARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in EA_return.loc[EA_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[3])*(i-parameter_0[3])/(sigma2)/2
        if i-parameter_0[3]<0:
            S = 1
        else:
            S = 0
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[4]*S*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[2]*sigma2
    return result

res_TGARCH = minimize(L_TGARCH, parameter_TGARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_TGARCH

Optimization terminated successfully.
         Current function value: -2455.610459
         Iterations: 545
         Function evaluations: 903


 final_simplex: (array([[1.05316049e-04, 1.28931569e-01, 5.88588294e-01, 2.96356267e-04,
        1.67099946e-02],
       [1.05316049e-04, 1.28931569e-01, 5.88588293e-01, 2.96356278e-04,
        1.67099940e-02],
       [1.05316049e-04, 1.28931569e-01, 5.88588293e-01, 2.96356271e-04,
        1.67099942e-02],
       [1.05316049e-04, 1.28931569e-01, 5.88588293e-01, 2.96356274e-04,
        1.67099943e-02],
       [1.05316049e-04, 1.28931568e-01, 5.88588293e-01, 2.96356265e-04,
        1.67099944e-02],
       [1.05316049e-04, 1.28931568e-01, 5.88588293e-01, 2.96356270e-04,
        1.67099947e-02]]), array([-2455.61045853, -2455.61045853, -2455.61045853, -2455.61045853,
       -2455.61045853, -2455.61045853]))
           fun: -2455.6104585282774
       message: 'Optimization terminated successfully.'
          nfev: 903
           nit: 545
        status: 0
       success: True
             x: array([1.05316049e-04, 1.28931569e-01, 5.88588294e-01, 2.96356267e-04,
       1.67099946e-02])

In [14]:
TGARCH_estimated_vol = []
sigma2 = vol*vol
for i in EA_return:
    if i-res_TGARCH.x[3]<0:
        S = 1
    else:
        S = 0
    sigma2 = res_TGARCH.x[0]+res_TGARCH.x[1]*(i-res_TGARCH.x[3])*(i-res_TGARCH.x[3])+res_TGARCH.x[4]*S*(i-res_TGARCH.x[3])*(i-res_TGARCH.x[3])+res_TGARCH.x[2]*sigma2
    TGARCH_estimated_vol.append(sigma2)
TGARCH_estimated_vol.pop
TGARCH_estimated_vol = pd.DataFrame(TGARCH_estimated_vol, index = EA_return.index, columns = ['TGARCH_estimated_vol'])
TGARCH_estimated_vol.mean()

TGARCH_estimated_vol    0.000383
dtype: float64

In [21]:
#np.sqrt(TGARCH_estimated_vol*252).to_pickle('Estimated Volatility/EA_TGARCH_estimated_vol.pkl')

## IGARCH

In [16]:
parameter_IGARCH = np.array([vol,0.66,EA_return_mean])#alfa_0,beta_1,miu


def L_IGARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in EA_return.loc[EA_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[2])*(i-parameter_0[2])/(sigma2)/2
        sigma2 = parameter_0[0]+(1-parameter_0[1])*(i-parameter_0[2])*(i-parameter_0[2])+parameter_0[1]*sigma2
    return result

res_IGARCH = minimize(L_IGARCH, parameter_IGARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_IGARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2429.644395
         Iterations: 312
         Function evaluations: 580


 final_simplex: (array([[5.10515608e-05, 6.84674599e-01, 1.30406132e-04],
       [5.10515606e-05, 6.84674600e-01, 1.30406127e-04],
       [5.10515606e-05, 6.84674600e-01, 1.30406119e-04],
       [5.10515607e-05, 6.84674600e-01, 1.30406125e-04]]), array([-2429.64439509, -2429.64439509, -2429.64439509, -2429.64439509]))
           fun: -2429.644395094239
       message: 'Optimization terminated successfully.'
          nfev: 580
           nit: 312
        status: 0
       success: True
             x: array([5.10515608e-05, 6.84674599e-01, 1.30406132e-04])

In [17]:
IGARCH_estimated_vol = []
sigma2 = vol*vol
for i in EA_return:
    sigma2 = res_IGARCH.x[0]+(1-res_IGARCH.x[1])*(i-res_IGARCH.x[2])*(i-res_IGARCH.x[2])+res_IGARCH.x[1]*sigma2
    IGARCH_estimated_vol.append(sigma2)
IGARCH_estimated_vol.pop
IGARCH_estimated_vol = pd.DataFrame(IGARCH_estimated_vol, index = EA_return.index, columns = ['IGARCH_estimated_vol'])
IGARCH_estimated_vol.mean()

IGARCH_estimated_vol    0.000542
dtype: float64

In [22]:
#np.sqrt(IGARCH_estimated_vol*252).to_pickle('Estimated Volatility/EA_IGARCH_estimated_vol.pkl')